In [2]:
import nltk 
import glob, os
import re
import string
import sys
import random
import xml.etree.ElementTree as ET
import xml.etree
import numpy
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from collections import Counter
from nltk.corpus import stopwords


from tqdm import tqdm_notebook as tqdm
from p_tqdm import p_map

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/chryssa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/chryssa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
##download parcit dir
os.system('wget -r -np -k -A tgz https://acl-arc.comp.nus.edu.sg/archives/acl-arc-160301-parscit/')

2

In [5]:
rootdir = 'acl-arc.comp.nus.edu.sg/archives/'
xmlDir = 'acl-arc.comp.nus.edu.sg/archives/acl-arc-160301-parscit/'
writeDirXML = 'acl-arc.comp.nus.edu.sg/archives/acl-arc-160301-parscit/processed/'

if not os.path.exists(writeDirXML):
    os.makedirs(writeDirXML)

In [7]:
def text_valid(sentence):
    lemmatizer = WordNetLemmatizer() 
    tokenized_text = nltk.word_tokenize(sentence)
    length = len(tokenized_text)
    alpha = 0
    nonAlpha = 0
    found = 0
    nonFound = 0
    upper = 0
    stop = 0
    stop_words = set(stopwords.words('english')) 
    
    for token in tokenized_text:
        if len(token)==1:
            if token.isalpha():
                alpha+=1
            else:
                nonAlpha+=1
        else:
            if(token.lower() in stop_words):
                stop+=1
            else:
                lemma = lemmatizer.lemmatize(token.lower())
                if len(wn.synsets(lemma))>0:
                    found+=1
                else:
                    if token.isalpha():
                        if not token.islower() and not token.isupper():
                            upper+=1
                        elif token.isupper():
                            upper+=1
                        else:
                            nonFound+=1
                    else:
                        nonAlpha+=1
    alphaS = alpha/length
    nonAlphaS = nonAlpha/length
    foundS = found/length
    nonFoundS = nonFound/length
    upperS = upper/length
    stopS = stop/length
    nv = False
    if(nonFoundS>0.3):
        nv = True
    if(foundS<0.11):
        nv = True
    if(alphaS>0.2):
        nv = True
    if nv:
        return False
    return True

In [8]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


In [ ]:
#XML processing
lemmatizer = WordNetLemmatizer() 
spunct = '!).:;>?\]|}'
counterS = 0 #parse written lines
counterF = 0 #parse processed files
counterE = 0 #parse procssing errors
for subdir, dirs, files in tqdm(os.walk(xmlDir)):
    for file in (files):
        if file.endswith('.xml'):
            x = os.path.join(subdir, file)
            counterF+=1
            fileSent = []
            fileSent_temp = []
            with open(x, 'r', encoding='utf-8', errors='ignore') as xmlFile:
                data=xmlFile.read()
                parser = ET.XMLParser()
                try:
                    xmlTree = ET.fromstring(data,parser)
                except:
                    print('error with XML parsing: '+ str(file))
                    counterE +=1
                
                # iterate and keep all titles
                for algos in xmlTree:
                    for algo in algos:
                        for var in algo:
                            #print(var.tag, var.attrib)
                            tit = var.find('title')
                            if not str(tit)=="None":
                                if not str(tit.text)=="NoneType":
                                    try:
                                        titl = tit.text.split("\n")
                                        sentBuffer = ''
                                        for line in titl:
                                            fixed = False
                                            if len(line)>1:
                                                if sentBuffer.strip().endswith('-'):
                                                    firstWord = line.split(" ")[0]
                                                    lemma = lemmatizer.lemmatize(firstWord.lower())
                                                    if not len(wn.synsets(lemma))>0:
                                                        wl = sentBuffer[0:len(sentBuffer)-2].split(" ")
                                                        w2 = wl[len(wl)-1]+firstWord
                                                        lemma2 = lemmatizer.lemmatize(w2.lower())
                                                        if len(wn.synsets(lemma2))>0:
                                                            sentBufferC = sentBuffer[0:len(sentBuffer)-2]+line
                                                            fixed = True        
                                                if not fixed:
                                                    sentBufferC = sentBuffer+' '+line
                                                sentBuffer = sentBufferC

                                        fileSent.append(sentBuffer.strip())
                                    except:
                                        print('error with title parsing: '+ str(tit) + ' for file: ' + str(file))
                                        counterE +=1
                
                # iterate bodytext
                for body in xmlTree.iter("bodyText"):
                    conf = body.get('confidence')
                    if float(conf)>0.6:
                        stext = body.text.split("\n")
                        sentBuffer = ''
                        for line in stext:
                            fixed = False
                            if len(line)>1:
                                if sentBuffer.strip().endswith('-'):
                                    firstWord = line.split(" ")[0]
                                    lemma = lemmatizer.lemmatize(firstWord.lower())
                                    if not len(wn.synsets(lemma))>0:
                                        wl = sentBuffer[0:len(sentBuffer)-2].split(" ")
                                        w2 = wl[len(wl)-1]+firstWord
                                        lemma2 = lemmatizer.lemmatize(w2.lower())
                                        if len(wn.synsets(lemma2))>0:
                                            sentBufferC = sentBuffer[0:len(sentBuffer)-2]+line
                                            fixed = True        
                                if not fixed:
                                    sentBufferC = sentBuffer+' '+line
                                sentBuffer = sentBufferC
                        if len(sentBuffer)>0:
                            fileSent_temp.append(sentBuffer.strip())
                            
                sentBuffer = ''
                for sent in fileSent_temp:
                    sentBuffer+=sent 
                    sentBuffer+= ' '
                    if sent.strip()[-1] in spunct:
                        fileSent.append(sentBuffer)
                        sentBuffer=''
                     
                wf = open(writeDirXML+file.split(".")[0]+"_sentences.txt", "w") 
                for line in fileSent:
                    sent_text = nltk.sent_tokenize(line) 
                    for sentence in sent_text:
                        if text_valid(sentence):
                            wf.write(sentence.strip() + '\n')
                            counterS+=1

print("processed: %d files and %d lines" % (counterF, counterS))
print("processing errors: %d " % counterE)